In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()


In [3]:
# zero revenue in "STATE_REVENUE" is missing data
useducation_df=useducation_df.replace(0, np.nan)
# Replace 'VI', 'U.S._VIRGIN_ISLANDS', as 'VIRGIN_ISLAND'
useducation_df=useducation_df.replace({'VI':'VIRGIN_ISLANDS', 'U.S._VIRGIN_ISLANDS':'VIRGIN_ISLANDS', \
                                       'AS': 'AMERICAN_SAMOA', 'BI':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BUREAU_OF_INDIAN_EDUCATIO':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BUREAU_OF_INDIAN_EDUCATION': 'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'BIE':'BUREAU_OF_INDIAN_AFFAIRS', \
                                       'DD': 'DEPARTMENT_OF_DEFENSE', \
                                       'DEPARTMENT_OF_DEFENSE_EDUCATION_ACTIVITY':'DEPARTMENT_OF_DEFENSE', \
                                       '^DOD.*$':'DEPARTMENT_OF_DEFENSE', \
                                       'GU':'GUAM', 'NORTHERN_MARIANAS':'NORTHERN_MARIANA_ISLANDS', \
                                       'PR':'PUERTO_RICO', 'COMMONWEALTH_OF_MARIANAS':'MARIANAS'})

useducation_df.loc[useducation_df['STATE'].str.contains('DOD'),'STATE']='DEPARTMENT_OF_DEFENSE'


In [4]:
# # Group by state
# useducation2_df = useducation_df.drop(columns = 'YEAR').groupby('STATE').mean()

# Replace by state mean

column_list = ['ENROLL','TOTAL_REVENUE','FEDERAL_REVENUE','STATE_REVENUE','LOCAL_REVENUE','TOTAL_EXPENDITURE', \
               'INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE', \
               'CAPITAL_OUTLAY_EXPENDITURE','GRADES_PK_G','GRADES_KG_G','GRADES_4_G','GRADES_8_G', 'GRADES_12_G', \
               'GRADES_1_8_G','GRADES_9_12_G','GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', \
               'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']


# useducation2_df[column_list]= useducation2_df.groupby("STATE").transform(lambda x: x.fillna(x.mean()))[column_list]

useducation2_df=useducation_df.copy()

for column in column_list:
    useducation2_df.loc[:, column].fillna(useducation2_df[column].mean(), inplace=True)

In [5]:
useducation2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1492 non-null float64
TOTAL_REVENUE                   1492 non-null float64
FEDERAL_REVENUE                 1492 non-null float64
STATE_REVENUE                   1492 non-null float64
LOCAL_REVENUE                   1492 non-null float64
TOTAL_EXPENDITURE               1492 non-null float64
INSTRUCTION_EXPENDITURE         1492 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1492 non-null float64
OTHER_EXPENDITURE               1492 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1492 non-null float64
GRADES_PK_G                     1492 non-null float64
GRADES_KG_G                     1492 non-null float64
GRADES_4_G                      1492 non-null float64
GRADES_8_G                      1

** TASK**
1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.
2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?
3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?
4. What is the correlation between the overall score variable and the 1st principal component?
5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

# Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [6]:
# score weighted by student number
useducation2_df['WEIGHTED_SCORE']=(useducation2_df.GRADES_4_G*0.5*(useducation2_df.AVG_MATH_4_SCORE+useducation2_df.AVG_READING_4_SCORE)+  \
                                   useducation2_df.GRADES_8_G*0.5*(useducation2_df.AVG_MATH_8_SCORE+useducation2_df.AVG_READING_8_SCORE))/ \
                                (useducation2_df.GRADES_4_G+useducation2_df.GRADES_8_G)


# What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [7]:
education_exp = useducation2_df.loc[:, ['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE', 'WEIGHTED_SCORE']]

In [8]:
education_exp.corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,WEIGHTED_SCORE
TOTAL_EXPENDITURE,1.000000,0.991347,0.993314,0.939463,0.932447,0.069725
INSTRUCTION_EXPENDITURE,0.991347,1.000000,0.976889,0.902964,0.896103,0.075448
SUPPORT_SERVICES_EXPENDITURE,0.993314,0.976889,1.000000,0.945724,0.917782,0.072963
OTHER_EXPENDITURE,0.939463,0.902964,0.945724,1.000000,0.916854,0.010777
CAPITAL_OUTLAY_EXPENDITURE,0.932447,0.896103,0.917782,0.916854,1.000000,0.025506
WEIGHTED_SCORE,0.069725,0.075448,0.072963,0.010777,0.025506,1.000000


INSTRUCTION_EXPENDITURE is more correlated to WEIGHTED_SCORE.  However, corr = 0.075 is barely correlated. 

# Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# build the matrix
X=education_exp[['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE', 'WEIGHTED_SCORE']]
Y=education_exp['WEIGHTED_SCORE']

# Standardize X
X = StandardScaler().fit_transform(X)
sklearn_pca = PCA(n_components=1)
education_exp['PCA_1'] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)


The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.79496584]


79% of total variance is explained by the first component.  

# What is the correlation between the overall score variable and the 1st principal component?

In [10]:
education_exp.corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,WEIGHTED_SCORE,PCA_1
TOTAL_EXPENDITURE,1.000000,0.991347,0.993314,0.939463,0.932447,0.069725,0.995313
INSTRUCTION_EXPENDITURE,0.991347,1.000000,0.976889,0.902964,0.896103,0.075448,0.977308
SUPPORT_SERVICES_EXPENDITURE,0.993314,0.976889,1.000000,0.945724,0.917782,0.072963,0.990719
OTHER_EXPENDITURE,0.939463,0.902964,0.945724,1.000000,0.916854,0.010777,0.962968
CAPITAL_OUTLAY_EXPENDITURE,0.932447,0.896103,0.917782,0.916854,1.000000,0.025506,0.954349
WEIGHTED_SCORE,0.069725,0.075448,0.072963,0.010777,0.025506,1.000000,0.066354
PCA_1,0.995313,0.977308,0.990719,0.962968,0.954349,0.066354,1.000000


The correlation between weighted score and the PCA_1 is 0.066, less than original 0.075.

# If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

We should choose INSTRUCTION_EXPENDITURE than the PCA_1 because the INSTRUCTION_EXPENDITURE is more correlated.  